# Summarize crashes per year per-town, Normalize for density

Our study seeks to perform temporal hotspot analysis on Massachusetts' towns' MassDOT Impact data. This will require creating a table of the pedestrian crash counts per Census Block Group, grouped into each individual year.

We also seek to normalize the counts by the population of the town. This will allow to adjust for density.

ArcGIS's "Emerging Hotspot Analysis", combined with the "Create Space Time Cube From Defined Locations (Space Time Pattern Mining)" to create the NetCDF file, will be used to do the temporal hotspot analysis. However, we are trying to normalize to population, which you can't do with a NetCDF in Arc.

We further complicate this by doing year-over-year normalizations based upon interpolated population counts for 2000, 2010, and 2020.

Our strategy to get around this will be to create dummy entries per tract, which represent a population-normalized count. That way, when the Emerging Hotspot Analysis slices by year, it will have a count variable that is relative to the population. That way, low population density's effect on the hotspot analysis will be minimized. Areas with no accidents will still produce this error, however, which an extension upon these methods will attempt to minimize as well. Changing the geographic level to work upon tracts rather than Block Groups may help.

In [1]:
# Environment
%cd /kt/data/massdot-impact

/kt/data/massdot-impact


In [2]:
import pandas
import geopandas
import numpy

## Load data

In [3]:
# Massachusetts Block Groups
ma_bg = geopandas.read_file('aux/CENSUS2010BLOCKGROUPS_POLY.shp').set_index('GEOID10',drop=False)

In [4]:
# The MassDot Impact pedestrian collisions
# then, drop the ones lacking coordinates - about 5% average data loss
mdi_ped = (pandas.read_csv('proc/mdi-ped.csv',
                           usecols = ['PERS_ID','CRASH_NUMB','PERS_NUMB','CRASH_DATETIME','X','Y'],
                           parse_dates = ['CRASH_DATETIME'],
                           low_memory=False)
           .dropna(subset=['X','Y'])
           .set_index('PERS_ID', drop=False))

mdi_ped_gdf = geopandas.GeoDataFrame(mdi_ped,
                                     geometry=geopandas.points_from_xy(mdi_ped.X, mdi_ped.Y),
                                     crs='EPSG:26986'
                                    )

Join `ma_bg` to `mdi_ped_gdf`

## Spatial join

In [5]:
mdi_ped_gdf_join = mdi_ped_gdf.sjoin(ma_bg,
                  how="inner",
                  predicate="intersects"
                 )

In [6]:
# See which columns we have
mdi_ped_gdf_join.columns

Index(['PERS_ID', 'CRASH_NUMB', 'CRASH_DATETIME', 'X', 'Y', 'PERS_NUMB',
       'geometry', 'index_right', 'STATEFP10', 'COUNTYFP10', 'TRACTCE10',
       'BLKGRPCE10', 'GEOID10', 'NAMELSAD10', 'MTFCC10', 'ALAND10', 'AWATER10',
       'INTPTLAT10', 'INTPTLON10', 'AREA_SQFT', 'AREA_ACRES', 'POP100_RE',
       'HU100_RE', 'LOGPL94171', 'LOGSF1', 'LOGACS0610', 'LOGSF1C',
       'SHAPE_AREA', 'SHAPE_LEN'],
      dtype='object')

In [7]:
mdi_ped_gdf_join = (mdi_ped_gdf_join[
     ['PERS_ID','CRASH_NUMB','PERS_NUMB','GEOID10','CRASH_DATETIME','geometry']]
                    .drop('PERS_ID', axis=1))

In [8]:
mdi_ped_gdf_join

,CRASH_NUMB,PERS_NUMB,GEOID10,CRASH_DATETIME,geometry
PERS_ID,,,,,
1481125.2,1481125,2,250173162012,2002-08-23 13:40:00,POINT (220801.568 921121.395)
1759419.2,1759419,2,250173162012,2004-07-17 21:33:00,POINT (221023.159 920876.883)
2085715.2,2085715,2,250173162012,2006-08-31 13:18:00,POINT (220773.434 921142.177)
2089675.2,2089675,2,250173162012,2006-09-18 15:14:00,POINT (220740.532 921139.240)
2142988.2,2142988,2,250173162012,2007-01-22 21:59:59,POINT (221313.466 921309.819)
...,...,...,...,...,...
4770216.2,4770216,2,250056541003,2019-10-18 12:04:59,POINT (251871.214 827284.910)
4791527.3,4791527,3,250110401002,2019-12-07 11:00:00,POINT (98447.711 937379.062)
4795807.8,4795807,8,250173143021,2019-12-30 14:16:59,POINT (220076.318 941577.558)


In [9]:
# Quick check that datetimes were parsed properly
mdi_ped_gdf_join.CRASH_DATETIME.iloc[0]

datetime.datetime(2002, 8, 23, 13, 40)

In [10]:
# shapefile test
mdi_ped_gdf_join

,CRASH_NUMB,PERS_NUMB,GEOID10,CRASH_DATETIME,geometry
PERS_ID,,,,,
1481125.2,1481125,2,250173162012,2002-08-23 13:40:00,POINT (220801.568 921121.395)
1759419.2,1759419,2,250173162012,2004-07-17 21:33:00,POINT (221023.159 920876.883)
2085715.2,2085715,2,250173162012,2006-08-31 13:18:00,POINT (220773.434 921142.177)
2089675.2,2089675,2,250173162012,2006-09-18 15:14:00,POINT (220740.532 921139.240)
2142988.2,2142988,2,250173162012,2007-01-22 21:59:59,POINT (221313.466 921309.819)
...,...,...,...,...,...
4770216.2,4770216,2,250056541003,2019-10-18 12:04:59,POINT (251871.214 827284.910)
4791527.3,4791527,3,250110401002,2019-12-07 11:00:00,POINT (98447.711 937379.062)
4795807.8,4795807,8,250173143021,2019-12-30 14:16:59,POINT (220076.318 941577.558)


## Counts per year

We first convert back to Pandas DF - We don't need the actual point data anymore as we're working with the BG GEOIDs. We can join to those GEOIDs in Arc to save filesize.

In [11]:
# Back to pandas - no need for geometries as we're going by Census Block Group
mdi_ped_bgjoin = pandas.DataFrame(mdi_ped_gdf_join.drop('geometry', axis=1))

# Convert CRASH_DATETIME to the year only - our analysis uses year slices
mdi_ped_bgjoin['year'] = mdi_ped_bgjoin.apply(lambda r: r.CRASH_DATETIME.year, axis=1)

mdi_ped_bg_counts = (mdi_ped_bgjoin
 .groupby('GEOID10')
 .agg(count=('year', 'value_counts'))
 .reset_index()
)

mdi_ped_bg_counts

,GEOID10,year,count
0,250010101001,2009,2
1,250010101001,2011,2
2,250010101001,2012,1
3,250010101001,2015,1
4,250010101001,2016,1
...,...,...,...
35141,250277614004,2019,1
35142,250277614005,2009,2
35143,250277614005,2010,2
35144,250277614006,2014,2


## Manual year pivot

Slow, a long pivot was not easy here, it could be easier in R tidyverse

In [12]:
bgs = sorted(set(ma_bg.index))

mdi_pivot = pandas.DataFrame(index=bgs)

for bg in bgs:
    bg_seg = mdi_ped_bg_counts[ mdi_ped_bg_counts.GEOID10 == bg ]

    # returns like :
    #                     GEOID10  year  count
    #     0  250010101001  2009      2
    #     1  250010101001  2011      2
    #     2  250010101001  2012      1
    #     3  250010101001  2015      1
    #     4  250010101001  2016      1
    
    mdi_pivot.loc[bg, bg_seg.year] = bg_seg['count'].tolist()
    
mdi_pivot = (mdi_pivot[sorted(mdi_pivot.columns)]
             .fillna(0)
             .astype(int))

# Fix index
mdi_pivot.index = mdi_pivot.index.astype(numpy.int64)

In [13]:
mdi_pivot

,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
250010101001,0,0,0,0,0,0,0,2,0,2,1,0,0,1,1,0,0,0
250010101002,0,1,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,1
250010101003,0,0,0,0,0,1,0,1,0,0,0,0,0,3,0,1,1,0
250010101004,0,0,3,1,1,2,2,3,1,0,9,1,2,2,2,2,8,3
250010101005,3,2,1,4,3,1,2,0,2,4,7,6,6,4,2,3,4,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250277614002,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0
250277614003,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1
250277614004,0,0,0,1,1,0,0,1,1,0,0,1,3,0,1,0,0,1
250277614005,0,0,0,0,0,0,0,2,2,0,0,0,0,0,0,0,0,0


In [14]:
numpy.amax(numpy.nan_to_num(mdi_pivot.values))

60

## Populations table

We need a table of the yearly populations for each tract to properly normalize.

In [15]:
# Force the abbreviation codes to string (preserve leading zeroes)
pops = pandas.read_csv('aux/nhgis0009_ts_geog2010_blck_grp.csv',
                      dtype={
                          'STATEA':str,'COUNTYA':str,'TRACTA':str,'BLCK_GRPA':str
                      })

# Filter to massachusetts only
pops = pops[ pops.STATE == 'Massachusetts' ]

# Create the GEOID10 we see on the MassGIS files for BGs
pops['GEOID10'] = pops.apply(
    lambda r: int(f"{r.STATEA}{r.COUNTYA}{r.TRACTA}{r.BLCK_GRPA}"),
    axis=1)
pops = pops.set_index(pops.GEOID10)

# Trim & rename
pops = pops[['CL8AA2000', 'CL8AA2010', 'CL8AA2020']]
pops.columns = ['2000','2010','2020']

# Cheap interpolation for years from 2001 to 2009 and 2011 to 2019
for x in range(1,10):
    pops[f"200{x}"] = pops.apply(lambda r: r['2000']*(1-x/10) + r['2010']*(x/10), axis=1)
for x in range(1,10):
    pops[f"201{x}"] = pops.apply(lambda r: r['2010']*(1-x/10) + r['2020']*(x/10), axis=1)

# Sort the columns again
pops = pops[list(sorted(pops.columns))]

# Make the columns ints
pops.columns = pops.columns.astype(int)

# Convert it to integer
pops = pops.astype(int)

### Drop missing from populations table

Necessary for division to work

In [16]:
print('checks for GEOID agreement')

pivot_not_pops = [v for v in
 mdi_pivot.index
 if v not in 
 pops.index]

print('GEOIDs in mdi_pivot that are not in pops')
print(pivot_not_pops)

pops_not_pivot = [v for v in
 pops.index
 if v not in 
 mdi_pivot.index]

print('GEOIDs in pops that are not in mdi_pivot')
print(pops_not_pivot)

print('Are the indexes the same?')
print(mdi_pivot.index.tolist() == pops.index.tolist())

checks for GEOID agreement
GEOIDs in mdi_pivot that are not in pops
[]
GEOIDs in pops that are not in mdi_pivot
[250019900000, 250059900000, 250079900000, 250099901000, 250199900000, 250239900030]
Are the indexes the same?
False


In [17]:
pops = pops.drop(pops_not_pivot)

Confirm the data lines up

In [18]:
pops.index.tolist() == mdi_pivot.index.tolist()

True

### Normalization of crash counts
       
We are going to normalize by dividing by the population at a given year: $\frac{\text{crashes}_{year} }{ \text{pop}_{year}}$

Then multiply by 100,000 to get a crashes-per-100K normalized statistic.

Normalize to number of crashes per person. We ignore division by 0 population, replacing with 0.

In [19]:
norm = numpy.nan_to_num(
                        numpy.divide(mdi_pivot.values, pops[mdi_pivot.columns].values),
                        posinf=0
)

norm_100k = numpy.array(norm*1e5, dtype='int64')

/tmp/ipykernel_1266282/4110935168.py:2: RuntimeWarning: divide by zero encountered in true_divide
  numpy.divide(mdi_pivot.values, pops[mdi_pivot.columns].values),
/tmp/ipykernel_1266282/4110935168.py:2: RuntimeWarning: invalid value encountered in true_divide
  numpy.divide(mdi_pivot.values, pops[mdi_pivot.columns].values),


## Clip

Visual spot-check shows that the data has extremely long-tail characteristics owing to low-population tracts. Airports in Boston and Yarmouth, government areas in Boston, Springfield, and Worcester, and commercial shopping zones exhibit this behavior. All within this class have a crashes-per-100K value above 1600, so we will hard-clip the table to a maximum of 1600.

In [20]:
norm_100k = numpy.clip(norm_100k, a_min=0, a_max=1600)

## Convert to data frame

In [21]:
mdi_pivot_norm_100k = pandas.DataFrame(index=mdi_pivot.index,
                                       data=norm_100k,
                                       columns=mdi_pivot.columns)
mdi_pivot_norm_100k.index.name = 'GEOID10'
mdi_pivot_norm_100k

,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
GEOID10,,,,,,,,,,,,,,,,,,
250010101001,0,0,0,0,0,0,0,262,0,254,123,0,0,114,111,0,0,0
250010101002,0,183,0,0,0,0,196,0,0,0,191,186,181,0,0,0,0,160
250010101003,0,0,0,0,0,168,0,173,0,0,0,0,0,495,0,161,159,0
250010101004,0,0,505,172,176,360,369,567,193,0,1600,172,332,321,311,300,1166,424
250010101005,394,270,138,571,441,151,312,0,333,660,1143,969,959,632,313,465,614,1065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250277614002,0,84,0,0,0,0,0,0,86,0,86,0,0,0,0,0,0,0
250277614003,0,0,0,0,0,100,0,0,0,0,0,0,0,0,0,93,0,91
250277614004,0,0,0,136,136,0,0,135,135,0,0,133,396,0,130,0,0,128


## Test export for previewing

In [22]:
mdi_pivot_norm_100k.to_csv('proc/mdi_pivot_norm100k.csv')

# Pivot long again (melt?)

In [23]:
mdi_pivot_norm_100k_long = mdi_pivot_norm_100k.copy()
mdi_pivot_norm_100k_long['GEOID10'] = mdi_pivot_norm_100k.index
mdi_pivot_norm_100k_long = mdi_pivot_norm_100k_long.melt(id_vars='GEOID10').set_index('GEOID10')
mdi_pivot_norm_100k_long.columns = ['year','crashes_per_100k']
mdi_pivot_norm_100k_long

,year,crashes_per_100k
GEOID10,,
250010101001,2002,0
250010101002,2002,0
250010101003,2002,0
250010101004,2002,0
250010101005,2002,394
...,...,...
250277614002,2019,0
250277614003,2019,91
250277614004,2019,128


Reattach `LOGSF1` ID value. ArcGIS needs this because `GEOID10` is too long to act as the ID string

In [41]:
ma_bg_logsf1 = pandas.DataFrame(ma_bg.LOGSF1)
ma_bg_logsf1.index = ma_bg_logsf1.index.astype('int64')

mdi_pivot_norm_100k_long = mdi_pivot_norm_100k_long.join(ma_bg_logsf1)

Convert to datetimes

In [49]:
from datetime import datetime
mdi_pivot_norm_100k_long['year_timestamp'] = mdi_pivot_norm_100k_long.apply(lambda r: datetime(year=r.year,month=1,day=1), axis=1)

In [50]:
mdi_pivot_norm_100k_long.sort_index(inplace=True)
mdi_pivot_norm_100k_long.to_csv('proc/mdi_pivot_norm_100k_long.csv')

Also long-pivot the original data for a non-normalized table

In [56]:
# Copy-paste of the above operations into one cell

mdi_pivot_long = mdi_pivot.copy()
mdi_pivot_long['GEOID10'] = mdi_pivot.index
mdi_pivot_long = mdi_pivot_long.melt(id_vars='GEOID10').set_index('GEOID10')
mdi_pivot_long.columns = ['year','crashes_total']

mdi_pivot_long['year_timestamp'] = mdi_pivot_long.apply(
    lambda r: datetime(year=r.year,month=1,day=1), axis=1
)
mdi_pivot_long.sort_index(inplace=True)
mdi_pivot_long = mdi_pivot_long.join(ma_bg_logsf1)
mdi_pivot_long.to_csv('proc/mdi_pivot_long.csv')

mdi_pivot_long

,year,crashes_total,year_timestamp,LOGSF1
GEOID10,,,,
250010101001,2002,0,2002-01-01,13423
250010101001,2017,0,2017-01-01,13423
250010101001,2016,1,2016-01-01,13423
250010101001,2019,0,2019-01-01,13423
250010101001,2004,0,2004-01-01,13423
...,...,...,...,...
250277614006,2014,2,2014-01-01,172116
250277614006,2015,1,2015-01-01,172116
250277614006,2016,0,2016-01-01,172116


# Create dummy events

In [294]:
dummy_events_table = pandas.DataFrame(columns=['GEOID10','datetime'])

from datetime import datetime

In [310]:
# change from mdi_pivot to the scaled copy
for geoid10,years in mdi_pivot.iterrows():
    years = years.dropna()
    crash_events = []
    for y,v in zip(years.index, years):
        v=int(v)
        dts = [datetime(year=y,month=1,day=1) for i in range(1,v+1)]
        crash_events += dts
    df_chunk = pandas.DataFrame({'GEOID10':[geoid10]*len(crash_events),
                                'datetime': crash_events})
    dummy_events_table = pandas.concat([dummy_events_table,df_chunk])

In [311]:
dummy_events_table

,GEOID10,datetime
0,250010101001,2009-01-01
1,250010101001,2009-01-01
2,250010101001,2011-01-01
3,250010101001,2011-01-01
4,250010101001,2012-01-01
...,...,...
2,250277614005,2010-01-01
3,250277614005,2010-01-01
0,250277614006,2014-01-01
1,250277614006,2014-01-01


Without the scaling, this is basically the join to BG with the datetime as just the year...

**Next**: check out what this looks like in arc

# Notes

 - This is probably better normalized to miles of road per tract
 - Opens the door to a model that considers miles of highway, pedestrian path, and surface roads
 - ArcGIS Pro Emerging Hotspot Analysis may allow for a weight to account for density attached to each point